In [ ]:
from flask import Flask, request, jsonify
import base64
import tempfile
import os
import sys
import logging
import json
import time
import threading
from datetime import datetime
from flask_cors import CORS
import paho.mqtt.client as mqtt

# Import the EnhancedTomatoDiseaseClient class from your existing code
from tomato_disease_client import EnhancedTomatoDiseaseClient

app = Flask(__name__)
# Enable CORS for all routes
CORS(app)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("tomato_backend.log"),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger("tomato-disease-backend")

# Load configuration from environment variables or use defaults
# This allows for easy configuration changes in different environments
SERVER_URL = os.environ.get("SERVER_URL", "http://localhost:5000")
API_KEY = os.environ.get("WEATHER_API_KEY", "YOUR_API_KEY_HERE")  # Set your key via env or replace with default
DEFAULT_LOCATION = os.environ.get("DEFAULT_LOCATION", "Coimbatore")
MQTT_BROKER = os.environ.get("MQTT_BROKER", "localhost")  # Default to localhost if not specified
MQTT_TOPIC_SENSORS = os.environ.get("MQTT_TOPIC_SENSORS", "sensor/data")
MQTT_REQUEST_TIMEOUT = int(os.environ.get("MQTT_REQUEST_TIMEOUT", "10"))  # Seconds to wait for sensor data

# Function to safely convert value to float
def safe_float_convert(value, default=None):
    """Convert a value to float safely, returning default if conversion fails"""
    if value is None:
        return default
    try:
        return float(value)
    except (ValueError, TypeError):
        logger.warning(f"Could not convert value '{value}' to float")
        return default

def get_sensor_readings():
    """Request and get the latest sensor readings via MQTT"""
    sensor_data = None
    received_data_event = threading.Event()
    
    def on_connect(client, userdata, flags, rc):
        logger.info(f"Connected to MQTT broker with result code {rc}")
        client.subscribe(MQTT_TOPIC_SENSORS)
        # Request new sensor readings by publishing to request topic
        client.publish("sensor/request", "send_data")
    
    def on_message(client, userdata, message):
        nonlocal sensor_data
        payload = message.payload.decode()
        
        try:
            raw_data = json.loads(payload)  # Parse JSON
            logger.info(f"Raw MQTT data received: {raw_data}")
            
            # Convert numeric values to float to avoid type comparison issues
            sensor_data = {
                "temperature": safe_float_convert(raw_data.get('temperature')),
                "humidity": safe_float_convert(raw_data.get('humidity')),
                "light_intensity": safe_float_convert(raw_data.get('light_intensity')),
                "soil_moisture": safe_float_convert(raw_data.get('soil_moisture'))
            }
            
            logger.info(f"Processed sensor data: Temperature={sensor_data['temperature']}°C, "
                      f"Humidity={sensor_data['humidity']}%, "
                      f"Light={sensor_data['light_intensity']}, "
                      f"Soil Moisture={sensor_data['soil_moisture']}%")
            received_data_event.set()  # Signal that data has been received
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding JSON from MQTT message: {e}, payload: {payload}")
    
    # Create MQTT client for this request
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_message = on_message
    
    try:
        client.connect(MQTT_BROKER, 1883, 60)
        client.loop_start()
        
        # Wait for sensor data with timeout
        if received_data_event.wait(timeout=MQTT_REQUEST_TIMEOUT):
            logger.info("Successfully received sensor data")
        else:
            logger.warning(f"Timeout waiting for sensor data after {MQTT_REQUEST_TIMEOUT} seconds")
            # Try to pull the most recent message (if any) before giving up
            time.sleep(1)  # Give a little more time for any in-flight messages
        
        client.loop_stop()
        client.disconnect()
        
        return sensor_data
    except Exception as e:
        logger.error(f"Failed to connect to MQTT broker: {str(e)}")
        return None
    
@app.route('/health', methods=['GET'])
def health_check():
    """Endpoint to check if the backend is running"""
    return jsonify({
        "status": "healthy", 
        "timestamp": datetime.now().isoformat(),
        "version": os.environ.get("APP_VERSION", "1.0.0")
    })

@app.route('/analyze', methods=['POST', 'OPTIONS'])
def analyze_disease():
    """Endpoint to analyze tomato disease from uploaded image"""
    # Handle OPTIONS request explicitly (for CORS preflight)
    if request.method == 'OPTIONS':
        response = jsonify({'status': 'ok'})
        response.headers.add('Access-Control-Allow-Origin', '*')
        response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
        response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE,OPTIONS')
        return response
        
    try:
        logger.info("Received analyze request")
        data = request.json
        
        # Check if required fields are provided
        if not data or 'image' not in data:
            logger.error("No image data provided in request")
            return jsonify({"error": "No image data provided"}), 400
        
        # Get location (use default if not provided)
        location = data.get('location', DEFAULT_LOCATION)
        logger.info(f"Processing request with location: {location}")
        
        # Decode base64 image
        image_data = data['image']
        if isinstance(image_data, str) and "base64," in image_data:
            # Handle data URLs (e.g., data:image/jpeg;base64,/9j/4AAQ...)
            image_data = image_data.split("base64,")[1]
        
        # Create a temporary file to save the image
        with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as temp_file:
            try:
                decoded_image = base64.b64decode(image_data)
                temp_file.write(decoded_image)
                temp_file_path = temp_file.name
                logger.info(f"Decoded image size: {len(decoded_image)} bytes")
            except Exception as decode_error:
                logger.error(f"Base64 decoding error: {str(decode_error)}")
                return jsonify({"error": f"Failed to decode image: {str(decode_error)}"}), 400
        
        # Initialize the client
        client = EnhancedTomatoDiseaseClient(SERVER_URL, API_KEY, location)
        
        # Send image to server and get prediction
        logger.info("Sending image to server for prediction...")
        prediction_result = client.send_image(temp_file_path)
        
        if prediction_result is None:
            os.unlink(temp_file_path)  # Clean up temp file
            return jsonify({"error": "Failed to get prediction from server"}), 500
        
        # Check if the image contains a valid tomato leaf
        if not prediction_result.get("is_valid_tomato", True):
            os.unlink(temp_file_path)  # Clean up temp file
            return jsonify({
                "error": "Not a tomato leaf",
                "detail": prediction_result.get("detail", "The image does not appear to contain a tomato leaf"),
                "is_valid_tomato": False
            }), 400
        
        # Process image analysis
        logger.info("Processing detailed disease analysis...")
        analysis_path, severity = client.process_image_analysis(temp_file_path, prediction_result)
        
        # Try to get sensor data first
        logger.info("Requesting sensor data...")
        sensor_data = get_sensor_readings()
        
        if sensor_data and sensor_data['temperature'] is not None and sensor_data['humidity'] is not None:
            logger.info("Using local sensor data for environmental analysis")
            current_weather = {
                'temp_c': sensor_data['temperature'],
                'humidity': sensor_data['humidity'],
                'light_intensity': sensor_data.get('light_intensity'),
                'soil_moisture': sensor_data.get('soil_moisture'),
                'condition': {'text': 'Based on sensor data'},
                'wind_kph': 0,  # Default values for missing fields
                'pressure_mb': 0,
                'precip_mm': 0
            }
            
            # For rainfall data, we still need to get from API
            _, rainfall_data, forecast_data = client.get_weather_data()
            data_source = "sensor"
        else:
            logger.info("Sensor data unavailable, using weather API")
            # Try with provided location first
            current_weather, rainfall_data, forecast_data = client.get_weather_data()
            
            # If that fails, try with default location
            if current_weather is None:
                logger.warning(f"Failed to get weather for {location}. Trying default location: {DEFAULT_LOCATION}")
                # Create a new client with the default location
                default_client = EnhancedTomatoDiseaseClient(SERVER_URL, API_KEY, DEFAULT_LOCATION)
                current_weather, rainfall_data, forecast_data = default_client.get_weather_data()
                # Add a note that we're using fallback location
                if current_weather:
                    current_weather['note'] = f"Using data from {DEFAULT_LOCATION} (fallback location)"
                    logger.info(f"Successfully retrieved weather data from default location: {DEFAULT_LOCATION}")
                
            data_source = "weather_api"
        
        if current_weather is None:
            os.unlink(temp_file_path)  # Clean up temp file
            return jsonify({"error": "Failed to fetch environmental data from both specified location and default location"}), 500
        
        # Generate recommendations
        logger.info("Generating comprehensive recommendations...")
        recommendations = client.generate_recommendations(
            prediction_result["predicted_class"],
            prediction_result["confidence"],
            current_weather
        )
        
        # Convert analysis image to base64 for sending to mobile app
        with open(analysis_path, "rb") as image_file:
            analysis_image = base64.b64encode(image_file.read()).decode('utf-8')
        
        # Clean up temporary files
        os.unlink(temp_file_path)
        
        # Prepare response
        response = {
            "detection": {
                "disease": recommendations['disease'],
                "confidence": recommendations['confidence'],
                "severity": recommendations.get('severity', "Unknown"),
                "severity_description": recommendations.get('severity_description', "Description not available"),
                "affected_area_percentage": severity,
                "is_valid_tomato": True,
                "tomato_confidence": prediction_result.get("tomato_confidence", 1.0)
            },
            "environment": {
                "temperature": current_weather['temp_c'],
                "humidity": current_weather['humidity'],
                "light_intensity": current_weather.get('light_intensity'),
                "soil_moisture": current_weather.get('soil_moisture'),
                "avg_rainfall_past_3days": sum(rainfall_data)/3 if rainfall_data else 0,
                "disease_risk_level": recommendations.get('risk_level', "Unknown"),
                "data_source": data_source,
                "location_used": DEFAULT_LOCATION if current_weather.get('note') else location
            },
            "recommendations": {
                "treatments": recommendations.get('treatments', []),
                "organic_treatments": recommendations.get('organic_treatments', []),
                "preventive_measures": recommendations.get('preventive_measures', []),
                "environmental_management": recommendations.get('environmental_recommendations', []),
                "treatment_schedule": recommendations.get('treatment_schedule', {})
            },
            "analysis_image": analysis_image,
            "all_probabilities": prediction_result.get("all_probabilities", []),
            "class_names": prediction_result.get("class_names", [])
        }
                
        logger.info(f"Analysis complete for {recommendations['disease']} using {data_source} data")
        return jsonify(response)
    
    except Exception as e:
        logger.error(f"Error processing request: {str(e)}", exc_info=True)
        return jsonify({"error": f"An error occurred: {str(e)}"}), 500

@app.route('/weather', methods=['POST', 'OPTIONS'])
def get_weather():
    """Endpoint to get weather data for a location"""
    # Handle OPTIONS request explicitly
    if request.method == 'OPTIONS':
        response = jsonify({'status': 'ok'})
        response.headers.add('Access-Control-Allow-Origin', '*')
        response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
        response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE,OPTIONS')
        return response
        
    try:
        data = request.json
        location = data.get('location', DEFAULT_LOCATION)
        
        logger.info(f"Fetching weather data for location: {location}")
        
        # First try to get sensor data
        sensor_data = get_sensor_readings()
        
        if sensor_data and sensor_data['temperature'] is not None and sensor_data['humidity'] is not None:
            logger.info("Using local sensor data for environmental data")
            
            # Initialize client just for weather API data (rainfall, forecast)
            client = EnhancedTomatoDiseaseClient(SERVER_URL, API_KEY, location)
            _, rainfall_data, forecast_data = client.get_weather_data()
            
            # Create response with sensor data
            response = {
                "current": {
                    "temperature": sensor_data['temperature'],
                    "humidity": sensor_data['humidity'],
                    "light_intensity": sensor_data.get('light_intensity'),
                    "soil_moisture": sensor_data.get('soil_moisture'),
                    "condition": "Data from local sensors",
                    "wind_kph": 0,  # Default values
                    "pressure_mb": 0,
                    "precipitation_mm": 0,
                    "data_source": "sensor"
                },
                "rainfall_history": rainfall_data,
                "forecast_summary": [
                    {
                        "date": day['date'],
                        "max_temp": day['day']['maxtemp_c'],
                        "min_temp": day['day']['mintemp_c'],
                        "avg_humidity": day['day']['avghumidity'],
                        "chance_of_rain": day['day']['daily_chance_of_rain'],
                        "condition": day['day']['condition']['text']
                    }
                    for day in forecast_data
                ] if forecast_data else []
            }
        else:
            logger.info("Sensor data unavailable, using weather API")
            # Try with the requested location first
            client = EnhancedTomatoDiseaseClient(SERVER_URL, API_KEY, location)
            current_weather, rainfall_data, forecast_data = client.get_weather_data()
            
            # If that fails, try with the default location
            location_used = location
            if current_weather is None:
                logger.warning(f"Failed to get weather for {location}. Trying default location: {DEFAULT_LOCATION}")
                default_client = EnhancedTomatoDiseaseClient(SERVER_URL, API_KEY, DEFAULT_LOCATION)
                current_weather, rainfall_data, forecast_data = default_client.get_weather_data()
                location_used = DEFAULT_LOCATION
                
            if current_weather is None:
                return jsonify({
                    "error": "Failed to fetch weather data from both specified location and default location"
                }), 500
                
            response = {
                "current": {
                    "temperature": current_weather['temp_c'],
                    "humidity": current_weather['humidity'],
                    "condition": current_weather['condition']['text'],
                    "wind_kph": current_weather['wind_kph'],
                    "pressure_mb": current_weather['pressure_mb'],
                    "precipitation_mm": current_weather['precip_mm'],
                    "soil_moisture": "Not available from weather API",
                    "data_source": "weather_api",
                    "location_used": location_used
                },
                "rainfall_history": rainfall_data,
                "forecast_summary": [
                    {
                        "date": day['date'],
                        "max_temp": day['day']['maxtemp_c'],
                        "min_temp": day['day']['mintemp_c'],
                        "avg_humidity": day['day']['avghumidity'],
                        "chance_of_rain": day['day']['daily_chance_of_rain'],
                        "condition": day['day']['condition']['text']
                    }
                    for day in forecast_data
                ] if forecast_data else []
            }
            
            # Add a note if we're using the fallback location
            if location_used != location:
                response["note"] = f"Using weather data from {location_used} (fallback location)"
        
        return jsonify(response)
        
    except Exception as e:
        logger.error(f"Error fetching weather data: {str(e)}", exc_info=True)
        return jsonify({"error": f"An error occurred: {str(e)}"}), 500

@app.route('/sensor_data', methods=['GET'])
def get_sensor_data():
    """Endpoint to get current sensor data status"""
    sensor_data = get_sensor_readings()
    
    if sensor_data and sensor_data['temperature'] is not None and sensor_data['humidity'] is not None:
        return jsonify({
            "status": "available",
            "timestamp": datetime.now().isoformat(),
            "data": sensor_data
        })
    else:
        return jsonify({
            "status": "unavailable",
            "message": "Could not retrieve sensor data"
        })

@app.route('/disease_info', methods=['GET', 'OPTIONS'])
def get_disease_info():
    """Endpoint to get information about all diseases"""
    # Handle OPTIONS request explicitly
    if request.method == 'OPTIONS':
        response = jsonify({'status': 'ok'})
        response.headers.add('Access-Control-Allow-Origin', '*')
        response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
        response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE,OPTIONS')
        return response
        
    try:
        # Initialize client to access the disease database
        client = EnhancedTomatoDiseaseClient(SERVER_URL, API_KEY, DEFAULT_LOCATION)
        
        # Return disease database in a structured format
        response = {disease: {
            "optimal_conditions": {
                "temperature_range": f"{info['optimal_temp'][0]}°C - {info['optimal_temp'][1]}°C",
                "humidity_range": f"{info['optimal_humidity'][0]}% - {info['optimal_humidity'][1]}%",
                "soil_moisture_range": f"{info.get('optimal_soil_moisture', [40, 60])[0]}% - {info.get('optimal_soil_moisture', [40, 60])[1]}%"
            },
            "severity_levels": info['severity_levels'],
            "treatments": info['treatments'],
            "preventive_measures": info['preventive_measures'],
            "organic_treatments": info['organic_treatments']
        } for disease, info in client.disease_database.items()}
        
        return jsonify(response)
        
    except Exception as e:
        logger.error(f"Error fetching disease information: {str(e)}", exc_info=True)
        return jsonify({"error": f"An error occurred: {str(e)}"}), 500

@app.route('/validate_leaf', methods=['POST', 'OPTIONS'])
def validate_leaf():
    """Endpoint to validate if an image contains a tomato leaf"""
    # Handle OPTIONS request explicitly
    if request.method == 'OPTIONS':
        response = jsonify({'status': 'ok'})
        response.headers.add('Access-Control-Allow-Origin', '*')
        response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
        response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE,OPTIONS')
        return response
    
    try:
        data = request.json
        
        # Check if required fields are provided
        if not data or 'image' not in data:
            logger.error("No image data provided in request")
            return jsonify({"error": "No image data provided"}), 400
        
        # Decode base64 image
        image_data = data['image']
        if isinstance(image_data, str) and "base64," in image_data:
            # Handle data URLs (e.g., data:image/jpeg;base64,/9j/4AAQ...)
            image_data = image_data.split("base64,")[1]
        
        # Create a temporary file to save the image
        with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as temp_file:
            try:
                decoded_image = base64.b64decode(image_data)
                temp_file.write(decoded_image)
                temp_file_path = temp_file.name
            except Exception as decode_error:
                logger.error(f"Base64 decoding error: {str(decode_error)}")
                return jsonify({"error": f"Failed to decode image: {str(decode_error)}"}), 400
        
        # Initialize the client and send image for leaf validation only
        client = EnhancedTomatoDiseaseClient(SERVER_URL, API_KEY, DEFAULT_LOCATION)
        
        # New method in EnhancedTomatoDiseaseClient to validate leaf only
        validation_result = client.validate_tomato_leaf(temp_file_path)
        
        # Clean up temporary file
        os.unlink(temp_file_path)
        
        # Return validation result
        if validation_result.get("is_valid_tomato", False):
            return jsonify({
                "is_valid_tomato": True,
                "confidence": validation_result.get("tomato_confidence", 0),
                "message": "Valid tomato leaf detected"
            })
        else:
            return jsonify({
                "is_valid_tomato": False,
                "confidence": validation_result.get("tomato_confidence", 0),
                "message": validation_result.get("detail", "Not a tomato leaf")
            }), 400
        
    except Exception as e:
        logger.error(f"Error validating leaf: {str(e)}", exc_info=True)
        return jsonify({"error": f"An error occurred: {str(e)}"}), 500

if __name__ == '__main__':
    # Check if important environment variables are set
    if API_KEY == "YOUR_API_KEY_HERE":
        logger.warning("No Weather API key provided. Set WEATHER_API_KEY environment variable for production use.")
    
    port = int(os.environ.get("PORT", 8000))
    debug_mode = os.environ.get("FLASK_DEBUG", "False").lower() == "true"
    
    logger.info(f"Starting Tomato Disease Backend on port {port}, debug mode: {debug_mode}")
    logger.info(f"Server URL: {SERVER_URL}")
    logger.info(f"MQTT Broker: {MQTT_BROKER}")
    logger.info(f"Default location: {DEFAULT_LOCATION}")
    
    # Set host to 0.0.0.0 to make it accessible from other devices on the network
    app.run(host='0.0.0.0', port=port, debug=debug_mode)

 * Serving Flask app '__main__'
 * Debug mode: off
2025-04-21 14:59:53,158 - werkzeug - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.107.180:8000
2025-04-21 14:59:53,159 - werkzeug - INFO - Press CTRL+C to quit
2025-04-21 14:59:56,734 - tomato-disease-backend - INFO - Connected to MQTT broker with result code 0


C:\Users\MATHAN\AppData\Local\Temp\ipykernel_27712\2661933387.py:87: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


2025-04-21 15:00:06,700 - tomato-disease-backend - WARNING - Timeout waiting for sensor data after 10 seconds
2025-04-21 15:00:07,837 - werkzeug - INFO - 192.168.107.39 - - [21/Apr/2025 15:00:07] "GET /sensor_data HTTP/1.1" 200 -
2025-04-21 15:01:06,936 - tomato-disease-backend - INFO - Connected to MQTT broker with result code 0
2025-04-21 15:01:16,944 - tomato-disease-backend - WARNING - Timeout waiting for sensor data after 10 seconds
2025-04-21 15:01:17,984 - werkzeug - INFO - 192.168.107.39 - - [21/Apr/2025 15:01:17] "GET /sensor_data HTTP/1.1" 200 -
2025-04-21 15:03:00,693 - tomato-disease-backend - INFO - Connected to MQTT broker with result code 0
2025-04-21 15:03:10,688 - tomato-disease-backend - WARNING - Timeout waiting for sensor data after 10 seconds
2025-04-21 15:03:11,777 - werkzeug - INFO - 192.168.107.39 - - [21/Apr/2025 15:03:11] "GET /sensor_data HTTP/1.1" 200 -
2025-04-21 15:06:16,300 - tomato-disease-backend - INFO - Connected to MQTT broker with result code 0
2025